In [ ]:
import re
import nltk
import numpy as np
import random
import string


nltk.download("punkt")
with open("movie_lines.txt/movie_lines.txt", "r", encoding="iso-8859-1") as archivo:
    data = archivo.read()

print(data[:5])

In [ ]:
prueba = "L2457 +++$+++ u16 +++$+++ m1 +++$+++ COLUMBUS +++$+++ Thank you."
pattern = re.match(r'L(.*\+\+\+\$\+\+\+){4} ', prueba)
print(pattern)
prueba_sin = re.split(r'L(.*\+\+\+\$\+\+\+){4} ', prueba)
print(prueba_sin)
prueba_sin2 = re.sub(r'L(.*\+\+\+\$\+\+\+){4} ',"",prueba)
print(prueba_sin2)

In [ ]:
data1 = re.sub(r'L(.*\+\+\+\$\+\+\+){4} ',"",data)
print(data1[:1000])

In [ ]:
tokens = nltk.word_tokenize(data1.lower())
input_sequence = []
output_sequence = []
for i in range(len(tokens)-1):
    input_sequence.append(tokens[i])
    output_sequence.append(tokens[i+1])
print(input_sequence[:30])
input_sequence1 = np.array([nltk.word_tokenize(sec) for sec in input_sequence], dtype=object)
input_sequence_list = input_sequence1.tolist()
output_sequence1 = np.array([nltk.word_tokenize(sec) for sec in output_sequence], dtype=object)
output_sequence_list = output_sequence1.tolist()
print(input_sequence1[:30])


In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding


model = Sequential()
model.add(Embedding(input_dim= len(tokens), output_dim=100))
model.add(LSTM(256))
model.add(Dense(len(tokens), activation= "softmax"))
model.compile(loss = "categorical_crossentropy", optimizer= "adam", metrics = ["accuracy"])



In [ ]:
model.fit(input_sequence_list, output_sequence_list, epochs=100)